## Ames Wang
## Homework 4
## 11/3

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By # used to import different ways to access data in the XML or HTML file
from selenium.webdriver.chrome.service import Service # no longer need to download a driver file, use service
from webdriver_manager.chrome import ChromeDriverManager # used to manage the Chrome driver to emulate a Chrome web browser
import time
import random

## Q1

In [ ]:
#Function to perform random scrolling
def random_scroll(browser, total_wait_time):
    total_height = browser.execute_script("return document.body.scrollHeight")
    scroll_steps = random.randint(3, 10)
    scroll_increment = total_height // scroll_steps
    time_per_step = total_wait_time / scroll_steps
    
    for step in range(scroll_steps):
        browser.execute_script(f"window.scrollBy(0, {scroll_increment});")
        random_wait = random.uniform(0.5 * time_per_step, 1.5 * time_per_step)
        time.sleep(random_wait)
    
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Initialize the WebDriver (make sure to have the appropriate driver installed)
browser = webdriver.Chrome()
teams = []
years = []
wins = []
losses = []

# Loop through the NHL pages (update range to the actual number of pages)
for i in range(1, 25):  # Adjust the range based on the actual number of pages
    url = f"https://www.scrapethissite.com/pages/forms/?page_num={i}"
    print(f"Scraping page: {url}")

    browser.get(url)

    total_wait_time = random.uniform(2, 20)
    random_scroll(browser, total_wait_time)

    # Extract the NHL team data
    team_elements = browser.find_elements(By.CLASS_NAME, "team")
    
    for team in team_elements:
        try:
            # Get the team name
            team_name = team.find_element(By.CLASS_NAME, "name").text
            teams.append(team_name)
            
            # Get the year
            year = team.find_element(By.CLASS_NAME, "year").text
            years.append(year)
            
            # Get the number of wins
            win = team.find_element(By.CLASS_NAME, "wins").text
            wins.append(win)
            
            # Get the number of losses
            loss = team.find_element(By.CLASS_NAME, "losses").text
            losses.append(loss)
        except Exception as e:
            print(f"Error extracting data for team: {e}")

    print(f"Page {i} done")

    # Add delay before moving to the next page
    time.sleep(random.uniform(2, 5))

# Close the browser
browser.close()

# Convert the data into a DataFrame
nhl_data = pd.DataFrame({
    "Team": teams,
    "Year": years,
    "Wins": wins,
    "Losses": losses
})

# Save the data as a CSV file
nhl_data.to_csv("nhl_scraped_raw.csv", index=False, encoding="utf-8")

print(f"DataFrame shape: {nhl_data.shape}")

Scraping page: https://www.scrapethissite.com/pages/forms/?page_num=1
Page 1 done
Scraping page: https://www.scrapethissite.com/pages/forms/?page_num=2
Page 2 done
Scraping page: https://www.scrapethissite.com/pages/forms/?page_num=3
Page 3 done
Scraping page: https://www.scrapethissite.com/pages/forms/?page_num=4
Page 4 done
Scraping page: https://www.scrapethissite.com/pages/forms/?page_num=5
Page 5 done
Scraping page: https://www.scrapethissite.com/pages/forms/?page_num=6
Page 6 done
Scraping page: https://www.scrapethissite.com/pages/forms/?page_num=7
Page 7 done
Scraping page: https://www.scrapethissite.com/pages/forms/?page_num=8
Page 8 done
Scraping page: https://www.scrapethissite.com/pages/forms/?page_num=9
Page 9 done
Scraping page: https://www.scrapethissite.com/pages/forms/?page_num=10
Page 10 done
Scraping page: https://www.scrapethissite.com/pages/forms/?page_num=11
Page 11 done
Scraping page: https://www.scrapethissite.com/pages/forms/?page_num=12
Page 12 done
Scraping p

## Q2

In [ ]:
#Step 1: Clean and transform nhl_data
# Clean up the data (remove \n and leading/trailing whitespace)
nhl_data['Team'] = nhl_data['Team'].str.replace('\n', '').str.strip()
nhl_data['Year'] = nhl_data['Year'].astype(str).str.replace('\n', '').str.strip()
nhl_data['Wins'] = nhl_data['Wins'].astype(str).str.replace('\n', '').str.strip()
nhl_data['Losses'] = nhl_data['Losses'].astype(str).str.replace('\n', '').str.strip()

# Convert Year, Wins, and Losses into integer data types (int64)
nhl_data['Year'] = nhl_data['Year'].astype('int64')
nhl_data['Wins'] = nhl_data['Wins'].astype('int64')
nhl_data['Losses'] = nhl_data['Losses'].astype('int64')

# Print the data types to verify
print(nhl_data.dtypes)

# Save the DataFrame to a CSV file with UTF-8 encoding
nhl_data.to_csv('nhl_scraped_raw.csv', index=False, encoding='utf-8')

print(f"DataFrame shape: {nhl_data.shape}")

## Q3

In [ ]:
nhl_expanded = pd.read_excel("nhl_2012-2021.xlsx")

# Check and rename columns if necessary
nhl_expanded.rename(columns={
    'Season': 'Year',  # Rename 'Season' to 'Year'
    'W': 'Wins',  # Replace 'ColumnE' with the actual column name for Wins
    'L': 'Losses',  # Replace 'ColumnF' with the actual column name for Losses
}, inplace=True)

# Clean up the data (remove trailing * from team names)
nhl_expanded['Team'] = nhl_expanded['Team'].str.rstrip('*').str.strip()

# Vertically merge DataFrames
nhl2 = pd.concat([nhl_data, nhl_expanded], ignore_index=True)

# Create a new calculated column for win percentage
nhl2['win%'] = nhl2['Wins'] / (nhl2['Wins'] + nhl2['Losses'])

# SKeep only the necessary columns
nhl2 = nhl2[['Team', 'Year', 'Wins', 'Losses', 'win%']]

# Print the shape of the final DataFrame
print(f"Shape of the final DataFrame: {nhl2.shape}")

# Print the first few rows of the final DataFrame to verify the contents
print(nhl2.head())

# Save the final DataFrame to a CSV file with UTF-8 encoding
nhl2.to_csv('nhl_final.csv', index=False, encoding='utf-8')

## Q4

In [ ]:
arena = pd.read_csv("nhl_hockey_arenas.csv")
arena.rename(columns={
    'Team Name': 'Team',  # Rename 'Team Name' to 'Team'
    'Arena Name': 'Arena',
    'Arena Location': 'Location',
    'Seating Capacity': 'Capacity'
}, inplace=True)

print("Unique team names in nhl2:", nhl2['Team'].unique())
print("Unique team names in arena:", arena['Team'].unique())

In [ ]:
# Correct any discrepancies
nhl2['Team'] = nhl2['Team'].replace('Mighty Ducks of Anaheim', 'Anaheim Ducks')
nhl2['Team'] = nhl2['Team'].replace('Seattle Kraken', 'Seattle Kracken')

In [ ]:
# Perform an inner join between nhl2 and arena
nhl3 = pd.merge(nhl2, arena[['Team', 'Arena', 'Location', 'Capacity']], on='Team', how='inner')

# Print the shape of the final DataFrame
print(f"Shape of the final DataFrame: {nhl3.shape}")

In [ ]:
# Save the final DataFrame to a CSV file with UTF-8 encoding
nhl3.to_csv('nhl_final.csv', index=False, encoding='utf-8')

## Q5

In [ ]:
# Save the final DataFrame to a CSV file with UTF-8 encoding
nhl3.to_csv('awang24_hockey_data.csv', index=False, encoding='utf-8')